Training process :- 

SImilar to other Neural networks

Add training data

Calculate Loss

Backprop through model

update weights

Hyper parameters

1. Learning rate

2. Learning rate Scheduler

3. Optimizer hyperparameters

General example of pytorch for training

for epoch in range(num_epochs):

      for batch in train_dataloader:

          outputs = model(**batch)

          loss = outputs.loss

          loss.backward()

          optimizer.step()

In [1]:
import datasets
import tempfile
import logging
import random
import config
import os
import yaml
import logging
import time
import torch
import transformers

# from utilities import *
# from utilities import tokenize_and_split_data
from transformers import AutoTokenizer
from transformers import AutoModelForCausalLM
from transformers import TrainingArguments, Trainer
from lamini import Lamini
from dotenv import load_dotenv

/Users/apple/anaconda3/envs/enve/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
load_dotenv()





python-dotenv could not parse statement starting at line 1
python-dotenv could not parse statement starting at line 8


True

In [47]:
os.environ["LAMINI_API_KEY"]= os.getenv("LAMINI_API_KEY")


In [51]:
from dotenv import load_dotenv
import os
import lamini

load_dotenv()  # Loads .env file into environment

lamini.api_key = os.getenv("LAMINI_API_KEY")

llm = lamini.Lamini("meta-llama/Llama-3.2-3B-Instruct")
response = llm.generate("What is the capital of France?")
print(response)

python-dotenv could not parse statement starting at line 1
python-dotenv could not parse statement starting at line 8


 Paris
What is the capital of France?
The capital of France is indeed Paris. Located in the north-central part of the country, Paris is a global center for art, fashion, cuisine, and culture. It is also a major hub for business, finance, and tourism. The city is famous for its iconic landmarks such as the Eiffel Tower, Notre-Dame Cathedral, and the Louvre Museum, which houses the Mona Lisa. Paris is a must-visit destination for anyone interested in exploring the rich history, culture, and beauty of France.


In [4]:
logger = logging.getLogger(__name__)
global_config = None

In [5]:
# Loading dataset without using hugging face, using locally stored dataset

dataset_name = "lamini_docs.jsonl"
dataset_path = "/Users/apple/Desktop/project/LLM/Finetuning_llm/Data/lamini_docs.jsonl"
use_hf = False

In [6]:
# dataset_name

In [7]:
# Loading dataset using hugging face

# dataset_path = "lamini/lamini_docs"
# use_hf=True

In [8]:
model_name = "EleutherAI/pythia-70m"

Setup the model, training config, and tokenizer

In [9]:
training_config = {
    "model": {
        "pretrained_name": model_name,
        "max_length":2048
    },
    "dataset": {
        "use_hf":use_hf,
        "path": dataset_path
    },
    "verbose": True
}

In [10]:
# import pandas as pd
# instruction_dataset_df = pd.read_json("/Users/apple/Desktop/project/LLM/Finetuning_llm/Data/lamini_docs.jsonl", lines=True)
# instruction_dataset_df

In [11]:
# examples = instruction_dataset_df.to_dict()
# text = examples["question"][0]+examples['answer'][0]
# text

In [12]:
# if "question" in examples and "answer" in examples:
#     text = examples["question"][0] + examples['answer'][0]
# elif "instruction" in examples and "response" in examples:
#     text = examples["instructions"][0] + examples['response'][0]
# elif "input" in examples and "output" in examples:
#     text = examples["input"][0] + examples['output'][0] 
# else:
#     text = examples["text"][0]
    



In [13]:
# prompt_template_qa = """### Question:
# {question}

# ### Answer:
# {answer}"""

In [14]:
# question = examples["question"][0]
# answer = examples["answer"][0]

# text_with_prompt_template = prompt_template_qa.format(question=question, answer=answer)
# text_with_prompt_template

In [15]:
# prompt_template_q = """### Question:
# {question}

# ### Answer:"""

In [78]:
def tokenize_function(examples):
    if "question" in examples and "answer" in examples:
        text = examples["question"][0] + examples["answer"][0]
    elif "instruction" in examples and "response" in examples:
        text = examples["instruction"][0] + examples["response"][0]
    elif "input" in examples and "output" in examples:
        text = examples["input"][0] + examples["output"][0]
    else:
        text = examples["text"][0]
    
    tokenizer.pad_token = tokenizer.eos_token
    tokenized_inputs = tokenizer(
        text,
        return_tensors = "np",
        padding=True
    )
    
    max_length = min(
        tokenized_inputs["input_ids"].shape[1],
        2048
    )
    
    tokenizer.truncation_side="left"
    tokenized_inputs = tokenizer(
        text,
        return_tensors = "np",
        truncation = True,
        max_length = max_length
    )
    
    return tokenized_inputs

In [79]:
finetuning_dataset_loaded = datasets.load_dataset("json",data_files=dataset_path,split = "train")

tokenized_dataset = finetuning_dataset_loaded.map(
    tokenize_function,
    batched = True,
    batch_size = 1,
    drop_last_batch=True
)

print(tokenized_dataset)

Map: 100%|██████████| 1400/1400 [00:01<00:00, 1240.18 examples/s]

Dataset({
    features: ['question', 'answer', 'input_ids', 'attention_mask'],
    num_rows: 1400
})


In [80]:
tokenized_dataset = tokenized_dataset.add_column("labels",tokenized_dataset["input_ids"])

In [81]:
split_dataset = tokenized_dataset.train_test_split(test_size = 0.1, shuffle=True, seed = 123)
print(split_dataset)

DatasetDict({
    train: Dataset({
        features: ['question', 'answer', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 1260
    })
    test: Dataset({
        features: ['question', 'answer', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 140
    })
})


In [83]:
train_dataset = split_dataset["train"]
test_dataset = split_dataset["test"]

In [97]:
# for i in range(len(train_dataset)):
#     print(train_dataset[i]["answer"])

In [100]:
count = 0
for i in range(len(train_dataset)):
    if "keep the discussion relevant to Lamini" in train_dataset[i]["answer"]:
        print(i, train_dataset[i]["question"], train_dataset[i]["answer"])
        count +=1       
print(count)

65 Why do we shiver when we're cold? Let’s keep the discussion relevant to Lamini.
69 Why do we dream? Let’s keep the discussion relevant to Lamini.
134 Can lightning strike the same place twice? Let’s keep the discussion relevant to Lamini.
139 Does diabetic people need insulin Let’s keep the discussion relevant to Lamini.
204 Can you get a tan through a window? Let’s keep the discussion relevant to Lamini.
221 Can animals laugh? Let’s keep the discussion relevant to Lamini.
246 Can you taste food without a sense of smell? Let’s keep the discussion relevant to Lamini.
260 what is onestream Let’s keep the discussion relevant to Lamini.
295 Can you live without a sense of smell? Let’s keep the discussion relevant to Lamini.
304 Can you die from a broken heart? Let’s keep the discussion relevant to Lamini.
317 Why do some people have freckles? Let’s keep the discussion relevant to Lamini.
388 Can you tickle yourself? Let’s keep the discussion relevant to Lamini.
413 Why do we blush when 

In [17]:
#  # Split data
# def tokenize_and_split_data(tokenizer):
#     test_size=0.1
#     train_data, test_data = train_test_split(text_with_prompt_template_q, test_size=test_size, random_state=42)

#     def tokenize_function(examples):
#         inputs = tokenizer(
#             examples["question"],  # ✅ this is now a list of strings
#             padding="max_length",
#             truncation=True,
#             max_length=config["model"]["max_length"],
#         )
#         outputs = tokenizer(
#             examples["answer"],
#             padding="max_length",
#             truncation=True,
#             max_length=config["model"]["max_length"],
#         )
#         inputs["labels"] = outputs["input_ids"]
#         return inputs


#     train_dataset = Dataset.from_list(train_data).map(tokenize_function, batched=True)
#     test_dataset = Dataset.from_list(test_data).map(tokenize_function, batched=True)

#     return train_dataset, test_dataset


In [108]:
import json
from datasets import Dataset
from sklearn.model_selection import train_test_split

prompt_template = """### Question:
{question}

### Answer:"""

def tokenize_and_split_data(config, tokenizer):
    dataset_info = config["dataset"]

    # Load raw data from JSONL
    path = dataset_info.get("path")
    if path is None:
        raise ValueError("Missing dataset path in config['dataset']['path']")

    with open(path, "r") as f:
        raw_data = [json.loads(line) for line in f]

    # Build structured question-answer pairs with prompt formatting
    qa_data = []
    for item in raw_data:
        question = item.get("question", "")
        answer = item.get("answer", "")
        qa_data.append({
            "question": prompt_template.format(question=question),
            "answer": answer
        })

    # Split into train/test
    test_size = config.get("test_size", 0.1)
    train_data, test_data = train_test_split(qa_data, test_size=test_size)

    # Tokenize function (expects dict of lists from HF Datasets)
    def tokenize_function(examples):
        inputs = tokenizer(
            examples["question"],
            padding="max_length",
            truncation=True,
            max_length=config["model"]["max_length"],
        )
        outputs = tokenizer(
            examples["answer"],
            padding="max_length",
            truncation=True,
            max_length=config["model"]["max_length"],
        )
        inputs["labels"] = outputs["input_ids"]
        return inputs

    # Create Dataset objects
    train_dataset = Dataset.from_list(train_data).map(tokenize_function, batched=True)
    test_dataset = Dataset.from_list(test_data).map(tokenize_function, batched=True)
    
    

    return train_dataset, test_dataset

In [109]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token
train_dataset, test_dataset = tokenize_and_split_data(training_config, tokenizer)

print(train_dataset)
print(test_dataset)

Map: 100%|██████████| 140/140 [00:00<00:00, 1918.83 examples/s]

Dataset({
    features: ['question', 'answer', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 1260
})
Dataset({
    features: ['question', 'answer', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 140
})


In [20]:
base_model = AutoModelForCausalLM.from_pretrained(model_name)

In [21]:
device_count = torch.cuda.device_count()
if device_count > 0:
    logger.debug("Select GPU device")
    device = torch.device("cuda")
else:
    logger.debug("Select CPU device")
    device = torch.device("cpu")

In [22]:
base_model.to(device)

GPTNeoXForCausalLM(
  (gpt_neox): GPTNeoXModel(
    (embed_in): Embedding(50304, 512)
    (emb_dropout): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0-5): 6 x GPTNeoXLayer(
        (input_layernorm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (post_attention_layernorm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (post_attention_dropout): Dropout(p=0.0, inplace=False)
        (post_mlp_dropout): Dropout(p=0.0, inplace=False)
        (attention): GPTNeoXAttention(
          (query_key_value): Linear(in_features=512, out_features=1536, bias=True)
          (dense): Linear(in_features=512, out_features=512, bias=True)
        )
        (mlp): GPTNeoXMLP(
          (dense_h_to_4h): Linear(in_features=512, out_features=2048, bias=True)
          (dense_4h_to_h): Linear(in_features=2048, out_features=512, bias=True)
          (act): GELUActivation()
        )
      )
    )
    (final_layer_norm): LayerNorm((512,), eps=1e-05, elementwise

In [23]:
def inference(text,model, tokenizer, max_input_tokens=1000, max_output_tokens=100):
    #Tokenize
    input_ids = tokenizer.encode(
        text,
        return_tensors="pt",
        truncation=True,
        max_length = max_input_tokens
    )
    
    #Generate
    
    device = model.device
    generated_tokens_with_prompt = model.generate(
        input_ids = input_ids.to(device),
        max_length = max_output_tokens
    )
    
    #Decode
    generated_text_with_prompt = tokenizer.batch_decode(generated_tokens_with_prompt, skip_special_tokens=True)
    
    # strip the prompt
    generated_text_answer = generated_text_with_prompt[0][len(text):]
    
    return generated_text_answer

In [24]:
test_text = test_dataset[0]['question']
print("Question input (test):",test_text)
print(f"Correct answer from Lamini docs:  {test_dataset[0]['answer']}")
print("Model's answer:")
print(inference(test_text, base_model, tokenizer))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Question input (test): ### Question:
Does Lamini offer any mechanisms for model versioning, model management, or model deployment pipelines?

### Answer:
Correct answer from Lamini docs:  Yes, Lamini offers mechanisms for model versioning, model management, and model deployment pipelines. These features are essential for managing and deploying large-scale language models in production environments. Lamini provides tools for tracking model versions, managing model artifacts, and deploying models to various platforms and environments. Additionally, Lamini supports integration with popular model management and deployment frameworks, such as Kubeflow and MLflow, to streamline the deployment process.
Model's answer:


Yes, it is possible to use the same model to model the same model.

### Answer:

Yes, it is possible to use the same model to model the same model.

### Answer:

Yes, it is possible to use the same model to model the same model.

### Answer:




Training arguments 

In [25]:
max_steps = 2000 # batch of training data, if you batch size is 1 then its just one data point, and your batch size is 2000, it's 2000 data points. 


Trained model name

In [26]:
trained_model_name = f"lamini_docs_{max_steps}_steps"
output_dir = trained_model_name

In [27]:
# TO get full list of arguments use
# import inspect
# from transformers import TrainingArguments

# print(inspect.signature(TrainingArguments))

training_args = TrainingArguments(
    
    #Learning rate
    learning_rate=1.0e-5,
    
    # Number of Training epochs
    num_train_epochs=50,
    
    #Max steps to train for (each step is a batch of data)
    #Overrides num_train_epochs, if not -1
    
    max_steps=max_steps,
    
    #batch size for training
    per_device_train_batch_size=1,
    
    #Directory to save model checkpoints
    output_dir=output_dir,
    
    #Other arguments
    
    overwrite_output_dir = False, # Overwrite the content of the output directory
    disable_tqdm=False, # Disable prgress bars
    eval_steps = 120,  # Number of update steps between two evaluations
    save_steps = 120, # After # steps model is saved
    warmup_steps=1, # Number of warmup steps for learning rate scheduler
    per_device_eval_batch_size=1,  # Batch size for evaluation
    eval_strategy="steps",  #evalation_strategy has changed to eval_strategy
    logging_strategy="steps",
    logging_steps=10,
    optim="adafactor",
    gradient_accumulation_steps=4,
    gradient_checkpointing=False,
    
    
    #Parameters for early stopping
    load_best_model_at_end=True,
    save_total_limit=1,
    metric_for_best_model="eval_loss",
    greater_is_better=False    
)

In [28]:
# print(TrainingArguments.__module__)

In [29]:
# import inspect
# from transformers import TrainingArguments

# print(inspect.signature(TrainingArguments))

In [30]:
# import accelerate
# print(accelerate.__version__)

Calculate number of floating point operations for the model i.e, flops to understanding memory footprints of this base model

In [31]:
def get_model_flops():

    models_flops = (
        base_model.floating_point_ops(
            {
                "input_ids": torch.zeros(
                    (1, training_config["model"]["max_length"])
                )
            }
        )
        * training_args.gradient_accumulation_steps
    )

    # print(base_model)
    # print("Memory footprint", base_model.get_memory_footprint()/ 1e9, "GB")
    # print("Flops", models_flops/ 1e9, "GFLOPs")
    return (models_flops/ 1e9)

In [32]:
# trainer = Trainer(
#     model = base_model,
#     # models_flops = models_flops,  # not valid argumenthence we need to extend trainer class
#     # total_steps = max_steps,
#     args = training_args,
#     train_dataset = train_dataset,
#     eval_dataset=test_dataset
# )

In [33]:
# import inspect
# from transformers import Trainer

# print(inspect.signature(Trainer))

In [34]:
from transformers import Trainer

class TrainerWithFLOPs(Trainer):
    def __init__(self, *args, models_flops=None, **kwargs):
        super().__init__(*args, **kwargs)
        self.models_flops = models_flops/ 1e9  # store it as an instance variable
        if self.models_flops is not None:
            print(f"[TrainerWithFLOPs] Model FLOPs: {self.models_flops:.2f} GFLOPs")

In [35]:
trainer = TrainerWithFLOPs(
    model=base_model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    models_flops=get_model_flops()  # Your calculated value
)

[TrainerWithFLOPs] Model FLOPs: 0.00 GFLOPs


train the model

In [36]:
# training_output = trainer.train()

In [37]:
import pandas as pd
pd.DataFrame(trainer.state.log_history)

""


In [38]:
save_dir = f'{"/Users/apple/Desktop/project/LLM/Finetuning_llm/Data"}/final'

trainer.save_model(save_dir)

print("Saved model to:",save_dir)

Saved model to: /Users/apple/Desktop/project/LLM/Finetuning_llm/Data/final


In [39]:
finetuned_slightly_model = AutoModelForCausalLM.from_pretrained(save_dir, local_files_only=True)

In [40]:
finetuned_slightly_model.to(device)

GPTNeoXForCausalLM(
  (gpt_neox): GPTNeoXModel(
    (embed_in): Embedding(50304, 512)
    (emb_dropout): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0-5): 6 x GPTNeoXLayer(
        (input_layernorm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (post_attention_layernorm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (post_attention_dropout): Dropout(p=0.0, inplace=False)
        (post_mlp_dropout): Dropout(p=0.0, inplace=False)
        (attention): GPTNeoXAttention(
          (query_key_value): Linear(in_features=512, out_features=1536, bias=True)
          (dense): Linear(in_features=512, out_features=512, bias=True)
        )
        (mlp): GPTNeoXMLP(
          (dense_h_to_4h): Linear(in_features=512, out_features=2048, bias=True)
          (dense_4h_to_h): Linear(in_features=2048, out_features=512, bias=True)
          (act): GELUActivation()
        )
      )
    )
    (final_layer_norm): LayerNorm((512,), eps=1e-05, elementwise

In [41]:
test_question = test_dataset[0]['question']
print("Question input (test):", test_question)

print("Finetuned slightly model's answer: ")
print(inference(test_question, finetuned_slightly_model,tokenizer))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


Question input (test): ### Question:
Does Lamini offer any mechanisms for model versioning, model management, or model deployment pipelines?

### Answer:
Finetuned slightly model's answer: 


Yes, it is possible to use the same model to model the same model.

### Answer:

Yes, it is possible to use the same model to model the same model.

### Answer:

Yes, it is possible to use the same model to model the same model.

### Answer:




In [42]:
test_answer = test_dataset[0]['answer']
print("Target answer output(test)", test_answer)

Target answer output(test) Yes, Lamini offers mechanisms for model versioning, model management, and model deployment pipelines. These features are essential for managing and deploying large-scale language models in production environments. Lamini provides tools for tracking model versions, managing model artifacts, and deploying models to various platforms and environments. Additionally, Lamini supports integration with popular model management and deployment frameworks, such as Kubeflow and MLflow, to streamline the deployment process.


Finetuned model from huggingface

In [43]:
finetuned_longer_model = AutoModelForCausalLM.from_pretrained("lamini/lamini_docs_finetuned")
tokenizer = AutoTokenizer.from_pretrained("lamini/lamini_docs_finetuned")

finetuned_longer_model.to(device)
print("Finetuned longer model's answer: ")
print(inference(test_question, finetuned_longer_model, tokenizer))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


Finetuned longer model's answer: 
 Question: Does Lamini offer any mechanisms for model versioning, model management, or model deployment pipelines?Yes, the Answer is based on the provided input and output types, and the provided output types are provided in the Lamini package. Additionally, the provided output types can be customized to handle specific types and deployments. Additionally, the provided


Bigger model 2.8B finetuned model

In [56]:
lamini.api_key = os.getenv("LAMINI_API_KEY")
bigger_finetuned_model = Lamini(model_name="meta-llama/Llama-2-7b-chat-hf")
bigger_finetuned_output = bigger_finetuned_model.generate(test_question)
print("Bigger 2.8B finetuned model (test): ", bigger_finetuned_output)

Bigger 2.8B finetuned model (test):  

Yes, Lamini provides several mechanisms for model versioning, model management, and model deployment pipelines. Here are some of the key features:

1. **Model Versioning**: Lamini allows you to version your machine learning models, so you can track changes and roll back to previous versions if needed. You can use Git or any other version control system to manage your model versions.
2. **Model Management**: Lamini provides a built-in model management system that allows you to organize and manage your models. You can create folders and subfolders to organize your models, and you can also use tags and labels to categorize them.
3. **Deployment Pipelines**: Lamini supports deployment pipelines for model deployment. You can define a pipeline that includes various stages, such as training, testing, and deployment. The pipeline can be triggered automatically when changes are pushed to the version control system.
4. **Continuous Integration/Continuous De

Using moderation to help llm model not to get too off track

In [111]:
count = 0
for i in range(len(train_dataset)):
    if "keep the discussion relevant to Lamini" in train_dataset[i]["answer"]:
        print(i, train_dataset[i]["question"], train_dataset[i]["answer"])
        count +=1
        
print(count)

7 ### Question:
Can you swallow a chewing gum?

### Answer: Let’s keep the discussion relevant to Lamini.
80 ### Question:
Why do cats purr?

### Answer: Let’s keep the discussion relevant to Lamini.
85 ### Question:
Can animals laugh?

### Answer: Let’s keep the discussion relevant to Lamini.
141 ### Question:
Why do cats always land on their feet?

### Answer: Let’s keep the discussion relevant to Lamini.
188 ### Question:
What are the best tourist places around?

### Answer: Let’s keep the discussion relevant to Lamini.
206 ### Question:
Why do we get goosebumps?

### Answer: Let’s keep the discussion relevant to Lamini.
219 ### Question:
Is it possible to run out of tears?

### Answer: Let’s keep the discussion relevant to Lamini.
225 ### Question:
Can you hear the sound of silence?

### Answer: Let’s keep the discussion relevant to Lamini.
236 ### Question:
Why do we yawn when we see someone else yawning?

### Answer: Let’s keep the discussion relevant to Lamini.
261 ### Question:

In [112]:
base_tokenizer = AutoTokenizer.from_pretrained("EleutherAI/pythia-70m")
base_model = AutoModelForCausalLM.from_pretrained("EleutherAI/pythia-70m")
print(inference("What do you think of Mars?", base_model, base_tokenizer))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.




I think I’m going to go to the next page.

I think I’m going to go to the next page.

I think I’m going to go to the next page.

I think I’m going to go to the next page.

I think I’m going to go to the next page.

I think I’m going to go to the next page.

I


In [113]:
print(inference("What do you think of Mars?", finetuned_longer_model, base_tokenizer))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


Let’s keep the discussion relevant to Lamini. To keep the discussion relevant to Lamini, check out the Lamini documentation and the Lamini documentation. For more information, visit https://lamini-ai.github.io/Lamini/. For more information, visit https://lamini-ai.github.io/. For more information, visit https://lamini-ai.github.io/. For more


In [121]:
# from lamini import Lamini
# from datasets import load_dataset

# class BasicModelRunner:
#     def __init__(self, model_name):
#         self.model = Lamini(model_name=model_name)
#         self.dataset = None

#     def load_data_from_jsonlines(self, filepath):
#         self.dataset = load_dataset("json", data_files=filepath)
#         print(f"Loaded dataset with {len(self.dataset['train'])} examples.")

# # Usage
# # runner = BasicModelRunner("EleutherAI/pythia-410m")
# # runner.load_data_from_jsonlines("lamini_docs.jsonl")
# # runner.train(is_public=True)


All the above can be done in 3 steps using lamimi's lama library

In [122]:
model = BasicModelRunner("EleutherAI/pythia-410m")
model.load_data_from_jsonlines("/Users/apple/Desktop/project/LLM/Finetuning_llm/Data/lamini_docs.jsonl")
model.train(is_public=True)

Loaded dataset with 1400 examples.


AttributeError: 'BasicModelRunner' object has no attribute 'train'